In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  4418k      0  0:00:18  0:00:18 --:--:-- 5848k


In [2]:
!rm -r aclImdb/train/unsup

In [3]:
import os, pathlib, shutil, random

import tensorflow as tf

from tensorflow import keras
from keras import layers

In [4]:
base_dir = pathlib.Path("aclImdb")

val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname, val_dir / category / fname)

In [5]:
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [6]:
max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [7]:
# inputs = keras.Input(shape=(None,), dtype="int64")
# embedded = tf.one_hot(inputs, depth=max_tokens)
# x = layers.Bidirectional(layers.LSTM(32))(embedded)
# x = layers.Dropout(0.5)(x)
# outputs = layers.Dense(1, activation="sigmoid")(x)

# model = keras.Model(inputs, outputs)

# model.compile(optimizer="rmsprop",
#               loss="binary_crossentropy",
#               metrics=["accuracy"])

# model.summary()

In [8]:
# callbacks = [
#     keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.tf",
#                                     save_best_only=True)
# ]
# model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

In [9]:
# model = keras.models.load_model("one_hot_bidir_lstm.tf")
# print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

In [10]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 bidirectional (Bidirection  (None, 64)                73984     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [11]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.tf",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 502s 796ms/step - loss: 0.5177 - accuracy: 0.7424 - val_loss: 0.4589 - val_accuracy: 0.7972
Epoch 2/10
625/625 [==============================] - 512s 820ms/step - loss: 0.3428 - accuracy: 0.8679 - val_loss: 0.3557 - val_accuracy: 0.8544
Epoch 3/10
625/625 [==============================] - 475s 760ms/step - loss: 0.2771 - accuracy: 0.9008 - val_loss: 0.3599 - val_accuracy: 0.8526
Epoch 4/10
625/625 [==============================] - 496s 793ms/step - loss: 0.2441 - accuracy: 0.9158 - val_loss: 0.3496 - val_accuracy: 0.8672
Epoch 5/10
625/625 [==============================] - 480s 768ms/step - loss: 0.1988 - accuracy: 0.9335 - val_loss: 0.3739 - val_accuracy: 0.8768
Epoch 6/10
625/625 [==============================] - 471s 753ms/step - loss: 0.1700 - accuracy: 0.9464 - val_loss: 0.4020 - val_accuracy: 0.8720
Epoch 7/10
625/625 [==============================] - 475s 761ms/step - loss: 0.1461 - accuracy: 0.9531 - val_loss: 0.3985 -

In [12]:
model = keras.models.load_model("embeddings_bidir_gru.tf")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 123s 155ms/step - loss: 0.3619 - accuracy: 0.8565
Test acc: 0.856
